In [40]:
import mne
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,recall_score, f1_score
from processing_functions import ppt_id, create_epochs, freq_ind, create_numeric_labels, relative_band_power
warnings.simplefilter('ignore')

In [2]:
from sklearn.svm import SVC

We follow the same arrangement as in kNearestNeighbor folder to get the dataset.

In [3]:
ppt_diagnostics = pd.read_csv('data/ds004504/participants.tsv',sep='\t')
target_labels = ppt_diagnostics['Group'].apply(create_numeric_labels).values
target_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [4]:
epoch_length = 2000
overlap_ratio = 0.5
freq_bands = np.array([0.5,4.0,8.0,13.0,25.0,45.0])
sample_freq = 500 #hertz

In [5]:
features = []
targets = []
for i in range(len(target_labels)):
    ppt = i + 1
    raw_data = mne.io.read_raw_eeglab('data/ds004504/derivatives/' + ppt_id(ppt)
                                  + '/eeg/' + ppt_id(ppt) + '_task-eyesclosed_eeg.set', preload = True)
    export = raw_data.to_data_frame()
    ppt_array = export.iloc[:,range(1,len(export.columns))].values
    del raw_data
    del export
    ppt_epochs = create_epochs(ppt_array,epoch_length,overlap_ratio)
    freqs, ppt_psd  = welch(ppt_epochs,fs=sample_freq, axis=1)
    ppt_rbp = relative_band_power(ppt_psd,freqs,freq_bands)
    ppt_rbp_reshaped = ppt_rbp.reshape((ppt_rbp.shape[0], -1))
    features += [ppt_rbp_reshaped]
    targets += [[target_labels[i]]*ppt_rbp.shape[0]]

We used the same shuffle method as in kNN file.

In [9]:
A_indices = list(range(36))
C_indices = list(i for i in range(36,65))
np.random.shuffle(A_indices)
np.random.shuffle(C_indices)

In [10]:
features = [features[A_indices[i]] for i in range(36)] + [features[C_indices[i]] for i in range(29)]
targets = [targets[A_indices[i]] for i in range(36)] + [targets[C_indices[i]] for i in range(29)]

In [38]:
np.concatenate((features[0])).shape

(39805,)

In the next part, we will leave 1 patient out for each of the 65 (36 AD+ 29 CN) patients and apply general SVM with kernel degree 2 polynomial for each case and calculate their accuracy, sensitivity, specificity and f1 scores.

In [42]:
cross_valid_acc = []
sensitivities = []
specificities = []
f1s = []
for i in range(65):
    # The following two arrays will store the target and features for the train set. We will append to it
    # as we go through various subjects.
    features_minus_i = ((features[0:65].copy()))
    del (features_minus_i)[i]
    
    targets_minus_i = ((targets[0:65].copy()))
    del (targets_minus_i)[i]
    
    train_X = np.concatenate(features_minus_i)
    train_y = np.concatenate(targets_minus_i)
    
    
    # The following two arrays will store target and features for the test set. 
    test_X = features[i]
    #np.concatenate((features[i],features[36+i]))
    test_y = targets[i]
    #np.concatenate((targets[i],targets[36+i]))
    
    #fitting the model
    
    svc = SVC(kernel='poly', degree=2, C=1000)
    svc.fit(train_X, train_y)
        
    # This stores the accuracy on the test data
    cross_valid_acc += [svc.score(test_X, test_y)]
    sensitivities += [recall_score(test_y, svc.predict(test_X), labels = [0,1], pos_label = 0)]
    specificities += [recall_score(test_y, svc.predict(test_X), labels = [0,1],pos_label = 1)]
    f1s += [f1_score(test_y, svc.predict(test_X),labels = [0,1],pos_label=0)]

In [43]:
cross_valid_acc

[0.6133651551312649,
 1.0,
 0.4024390243902439,
 0.9168591224018475,
 0.32608695652173914,
 0.5985037406483791,
 0.9762470308788599,
 0.9938144329896907,
 0.8777506112469438,
 0.9016393442622951,
 0.4694835680751174,
 0.9852941176470589,
 1.0,
 0.9006211180124224,
 0.7129186602870813,
 0.9013157894736842,
 0.07038123167155426,
 0.9814814814814815,
 0.6772727272727272,
 0.5478841870824054,
 0.9895287958115183,
 0.9285714285714286,
 0.71875,
 0.9880952380952381,
 0.8078602620087336,
 0.6266318537859008,
 0.5206971677559913,
 0.8176638176638177,
 0.549367088607595,
 0.326984126984127,
 0.9697986577181208,
 0.9392405063291139,
 0.7783375314861462,
 0.8326180257510729,
 1.0,
 0.7723577235772358,
 0.08418367346938775,
 0.995475113122172,
 0.12285012285012285,
 0.2625,
 0.4642857142857143,
 0.9154228855721394,
 0.9319899244332494,
 0.40848806366047746,
 0.9048843187660668,
 0.897172236503856,
 0.9881796690307328,
 0.9482758620689655,
 0.6384039900249376,
 0.9173333333333333,
 0.95208333333333

In [50]:
sensitivities

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.08418367346938775,
 0.995475113122172,
 0.12285012285012285,
 0.2625,
 0.4642857142857143,
 0.9154228855721394,
 0.9319899244332494,
 0.40848806366047746,
 0.9048843187660668,
 0.897172236503856,
 0.9881796690307328,
 0.9482758620689655,
 0.6384039900249376,
 0.9173333333333333,
 0.9520833333333333,
 0.39951573849878935,
 0.6807387862796834,
 0.44144144144144143,
 0.9952038369304557,
 0.8693333333333333,
 0.39847715736040606,
 0.4584450402144772,
 0.8012170385395537,
 0.6715176715176715,
 0.7712264150943396,
 0.4693396226415094,
 0.7029478458049887,
 0.7266514806378133,
 0.7467700258397932]

In [51]:
specificities

[0.6133651551312649,
 1.0,
 0.4024390243902439,
 0.9168591224018475,
 0.32608695652173914,
 0.5985037406483791,
 0.9762470308788599,
 0.9938144329896907,
 0.8777506112469438,
 0.9016393442622951,
 0.4694835680751174,
 0.9852941176470589,
 1.0,
 0.9006211180124224,
 0.7129186602870813,
 0.9013157894736842,
 0.07038123167155426,
 0.9814814814814815,
 0.6772727272727272,
 0.5478841870824054,
 0.9895287958115183,
 0.9285714285714286,
 0.71875,
 0.9880952380952381,
 0.8078602620087336,
 0.6266318537859008,
 0.5206971677559913,
 0.8176638176638177,
 0.549367088607595,
 0.326984126984127,
 0.9697986577181208,
 0.9392405063291139,
 0.7783375314861462,
 0.8326180257510729,
 1.0,
 0.7723577235772358,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [52]:
f1s

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.15529411764705883,
 0.9977324263038548,
 0.2188183807439825,
 0.4158415841584159,
 0.6341463414634146,
 0.9558441558441558,
 0.9647979139504563,
 0.5800376647834274,
 0.9500674763832657,
 0.94579945799458,
 0.9940546967895363,
 0.9734513274336283,
 0.7792998477929984,
 0.956884561891516,
 0.975453575240128,
 0.5709342560553633,
 0.8100470957613816,
 0.6125,
 0.9975961538461539,
 0.9300998573466476,
 0.5698729582577132,
 0.6286764705882352,
 0.8896396396396397,
 0.8034825870646767,
 0.8708388814913448,
 0.6388443017656501,
 0.8255659121171771,
 0.841688654353562,
 0.8550295857988166]

In [46]:
np.mean(cross_valid_acc),np.std(cross_valid_acc)

(0.7228340645878397, 0.25264719891606324)

In [47]:
np.mean(sensitivities), np.std(sensitivities)

(0.3009900565321345, 0.3794365147943596)

In [48]:
np.mean(specificities), np.std(specificities)

(0.4218440080557052, 0.4168935619863905)

In [49]:
np.mean(f1s), np.std(f1s)

(0.34372830588471975, 0.4116251118135292)